# 层次化索引

In [3]:
import pandas as pd
import numpy as np

层次化索引能在一个轴上拥有多个（两个以上）索引级别

In [5]:
data = pd.Series(np.random.randn(9),index = [['a','a','a','b','b','c','c','d','d'],[1,2,3,1,3,1,2,2,3]])
data                                #由列表或数组组成的列表（嵌套列表）作为索引

a  1    0.395180
   2   -0.992775
   3    0.535566
b  1   -1.021456
   3   -0.593889
c  1    0.691563
   2   -0.119980
d  2   -0.979655
   3    0.608484
dtype: float64

In [6]:
data.index

MultiIndex(levels=[['a', 'b', 'c', 'd'], [1, 2, 3]],
           codes=[[0, 0, 0, 1, 1, 2, 2, 3, 3], [0, 1, 2, 0, 2, 0, 1, 1, 2]])

对于一个层次化索引的对象，可以使用所谓的部分索引，使用它选取数据子集的操作更简单：

In [7]:
data['b']

1   -1.021456
3   -0.593889
dtype: float64

In [9]:
data[1]

-0.9927749608780895

In [10]:
data['b':'c']  #标签索引，包含尾项

b  1   -1.021456
   3   -0.593889
c  1    0.691563
   2   -0.119980
dtype: float64

In [12]:
data.loc[['b','d']]      #loc[]

b  1   -1.021456
   3   -0.593889
d  2   -0.979655
   3    0.608484
dtype: float64

还可以在‘内层’进行选取：

In [15]:
data.loc[:,2]   #在标签abcd中取标签为2的值

a   -0.992775
c   -0.119980
d   -0.979655
dtype: float64

层次化索引在数据重塑和基于分组的操作（如透视表生成）中扮演着重要的角色。例如，可以通过unstack方法将这段数据重新安排到一个DataFrame中：

In [16]:
data.unstack()    #即将层次化索引的Series生成为DataFrame

,1,2,3
a,0.395180,-0.992775,0.535566
b,-1.021456,NaN,-0.593889
c,0.691563,-0.119980,NaN
d,NaN,-0.979655,0.608484


unstack的逆运算是stack：

In [17]:
data.unstack().stack()

a  1    0.395180
   2   -0.992775
   3    0.535566
b  1   -1.021456
   3   -0.593889
c  1    0.691563
   2   -0.119980
d  2   -0.979655
   3    0.608484
dtype: float64

对于一个DataFrame，每条轴都可以有分层索引：

In [4]:
frame = pd.DataFrame(np.arange(12).reshape((4,3)),index = [['a','a','b','b'],[1,2,1,2]],columns = [['Ohio','Ohio','Colarado'],['Red','Red','Green']])
frame

Ohio     Colarado
     Red Red    Green
a 1    0   1        2
  2    3   4        5
b 1    6   7        8
  2    9  10       11

In [ ]:
各层都可以有名字（可以是字符串，也可以是别的python对象）。如果指定了名称，他们就会显示出来

In [5]:
frame.index.names = ['Key1','Key2']
frame.columns.names = ['state','color']
frame   #要区分索引名state、color与行标签Key2

state     Ohio     Colarado
color      Red Red    Green
Key1 Key2                  
a    1       0   1        2
     2       3   4        5
b    1       6   7        8
     2       9  10       11

有了部分列索引，可以轻松选取列分组：

In [6]:
frame['Ohio']

color      Red  Red
Key1 Key2          
a    1       0    1
     2       3    4
b    1       6    7
     2       9   10

可以单独创建MultiIndex然后复用。上面那个DataFrame中的（带有分级名称）列可以这样创建：

In [7]:
m_columns = pd.MultiIndex.from_arrays([['Ohio','Ohio','Colorada'],['Green','Red','Green']],names=['state','color'])
m_columns

MultiIndex(levels=[['Colorada', 'Ohio'], ['Green', 'Red']],
           codes=[[1, 1, 0], [0, 1, 0]],
           names=['state', 'color'])

In [10]:
m_index = pd.MultiIndex.from_arrays([['a','a','b','b'],[1,2,1,2]],names = ['Key1','Key2'])
m_index

MultiIndex(levels=[['a', 'b'], [1, 2]],
           codes=[[0, 0, 1, 1], [0, 1, 0, 1]],
           names=['Key1', 'Key2'])

In [11]:
#复用
frame2 = pd.DataFrame(np.arange(12).reshape((4,3)),
                      index = m_index,
                      columns = m_columns)
frame2

state      Ohio     Colorada
color     Green Red    Green
Key1 Key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

# 重排与分级排序

重新调整某条轴上各级别的顺序，或根据指定级别上的值对数据进行排序。swaplevel接受两个级别编号或名称，
并返回一个互换了级别的新对象（数据不发生变化）

In [15]:
frame.swaplevel('Key1','Key2')

state     Ohio     Colarado
color      Red Red    Green
Key2 Key1                  
1    a       0   1        2
2    a       3   4        5
1    b       6   7        8
2    b       9  10       11

而 sort_index则根据单个级别中的值对数据进行排序。交换级别时，常常也会用到sort_index，这样最终结果就会按照指定顺序进行字母排序了：

In [17]:
frame.sort_index(level=1)   #level = 1  表示将行索引的第二列进行重排序

state     Ohio     Colarado
color      Red Red    Green
Key1 Key2                  
a    1       0   1        2
b    1       6   7        8
a    2       3   4        5
b    2       9  10       11

In [25]:
frame.swaplevel(0,1).sort_index(level=0)   #swaplevel(0,1),第一列与第二列行标签重新排序
                                          #.sort_index（level=0） 再将第一列行标签中的值进行排序

state     Ohio     Colarado
color      Red Red    Green
Key2 Key1                  
1    a       0   1        2
     b       6   7        8
2    a       3   4        5
     b       9  10       11

# 根据级别汇总统计

许多对DataFrame和Series的描述和汇总都有一个level选项，它用于指定在某条轴上求和的级别。
再以上面那个DataFrame为例，我们可以根据行或者列上的级别来进行求和

In [28]:
frame.sum(level = 'Key2')

state Ohio     Colarado
color  Red Red    Green
Key2                   
1        6   8       10
2       12  14       16

In [30]:
frame.sum(level = 'color',axis = 1)

color      Red  Green
Key1 Key2            
a    1       1      2
     2       7      5
b    1      13      8
     2      19     11

# 使用DataFrame的列进行索引

想要将DataFrame的一个或多个列当做行索引来用，或者可能希望将行索引变成DataFrame的列：

In [33]:
frame = pd.DataFrame({'a':range(7),'b':range(7,0,-1),'c':['one','one','one','two',
        'two','two','two'],'d':[0,1,2,0,1,2,3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


DataFrame的set_index函数会将其中一个或多个列转换为行索引，创建一个新的DataFrame

In [36]:
frame2 = frame.set_index(['c','d'])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

默认情况下，那些列会从DataFrame中移除，但也可以保留下来：

In [37]:
frame.set_index(['c','d'],drop = False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

reset_index的功能跟set_index刚好相反，层次化索引的级别将会被转移到列里面：

In [38]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


# 2.合并数据集

1.pandas.merge可根据一个或多个键将不同的DataFrame中的行连接起来（取列索引中共同的值）

In [59]:
df1 = pd.DataFrame({'key':['b','b','a','c','a','a','b'],'data1':range(7)})
df2 = pd.DataFrame({'key':['a','b','d'],'data2':range(3)})
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [60]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


In [61]:
pd.merge(df1,df2)  #df1的key∩df2的key = b，a 
                   # 这个多对一的合并 ，即df1有多个被标记a和b的行，而df2中key列每个值只有一行

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


pd.merge（df1，df2）并没有指明要用哪个列进行连接。如果没有指定，merge就会将
重叠的列的列名当做键。不过，最好还是明确指定一下(on = )：

In [42]:
pd.merge(df1,df2,on = 'key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


如果两个对象的列名不同，也可以分别进行指定：

In [43]:
df3 = pd.DataFrame({'1key':['b','b','a','c','a','a','b'],'data1':range(7)})
df4 = pd.DataFrame({'2key':['a','b','d'],'data2':range(3)})
df3

,1key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [44]:
df4

,2key,data2
0,a,0
1,b,1
2,d,2


In [46]:
pd.merge(df3,df4,left_on ='1key',right_on = '2key' )

,1key,data1,2key,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


发现，c、d及相关的数据消失了。默认情况下，merge做的是‘内连接’，即取键的交集。

In [50]:
pd.merge(df1,df2,how='outer',on = 'key')  #'outer'取的是键的并集，组合了左右连接

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


In [52]:
pd.merge(df1,df2,how='left',on = 'key')  #how = left使用左表中的所有键
                                        # how = right 使用右表中所有的键   
                                        #  inner 使用两个表中都有的键  

,key,data1,data2
0,b,0,1.0
1,b,1,1.0
2,a,2,0.0
3,c,3,NaN
4,a,4,0.0
5,a,5,0.0
6,b,6,1.0


多对多的合并

In [63]:
df1 = pd.DataFrame({'key':['b','b','a','c','a','b'],'data1':range(6)})
df2 = pd.DataFrame({'key':['a','b','a','b','d'],'data2':range(5)})
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [64]:
df2

,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


In [65]:
pd.merge(df1,df2,how='left',on = 'key')   #这是多对多的合并：df1的a和b有多行，df2的a、b也有多行
                #多对多产生的是行的笛卡尔积。左边的DataFrame有3个‘b’行，右边有2个，则结果有6个‘b’行

,key,data1,data2
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
5,a,2,2.0
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


#连接方式只影响出现在结果中的不同的键的值（inner）：

In [67]:
pd.merge(df1,df2,how='inner')  #inner取两个表中都有的键 即 交集∩

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,b,5,1
5,b,5,3
6,a,2,0
7,a,2,2
8,a,4,0
9,a,4,2


要根据多个键进行合并，传入一个由列名组成的列表即可：

In [69]:
left = pd.DataFrame({'key1':['foo','foo','bar'],'key2':['one','two','one'],'lval':[1,2,3]})
right = pd.DataFrame({'key1':['foo','foo','bar','bar'],'key2':['one','one','one','two'],
                     'rval':[4,5,6,7]})
left

,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


In [70]:
right

,key1,key2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


In [71]:
pd.merge(left,right,on = ['key1','key2'])

,key1,key2,lval,rval
0,foo,one,1,4
1,foo,one,1,5
2,bar,one,3,6


In [72]:
pd.merge(left,right,on = ['key1','key2'],how = 'outer') #在进行列-列连接时，DataFrame对象中的索引会被丢弃

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


处理重复列名，merge有一个suffixes选项，用于指定附加到左右两个DataFrame对象的重叠列名上的字符串：

In [73]:
pd.merge(left,right,on='key1')  #多对多  笛卡尔积

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [74]:
pd.merge(left,right,on='key1',suffixes=('_left','_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


# 索引上的合并

有时候，DataFrame中的连接键位于其索引中。在这种情况下，可以传入left_index=True或right_index=True（或两个都传）以说明索引应该被用作连接键：

In [76]:
left1 = pd.DataFrame({'key1':['a','b','a','a','b','c'],'value':range(6)})
right1 = pd.DataFrame({'group_val':[3.5,7]},index = ['a','b'])
left1

,key1,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [77]:
right1

,group_val
a,3.5
b,7.0


In [81]:
pd.merge(left1,right1,left_on = 'key1',right_index = True) #必须同时都规定left和right

,key1,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


merge默认的是inner：取连接键的交集，可以通过外连接的方式得到它们的的并集：

In [83]:
pd.merge(left1,right1,left_on = 'key1',right_index = True,how = 'outer')

,key1,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


层次化索引数据的合并

In [85]:
lefth = pd.DataFrame({'key1':['Ohio','Ohio','Ohio','Nevada','Nevada'],'key2':[2000,2001,2002,2001,2002],'data':np.arange(5.)})
righth = pd.DataFrame(np.arange(12).reshape((6,2)),index = [['Nevada','Nevada','Ohio','Ohio','Ohio','Ohio'],[2001,2000,2000,2000,2001,2002]],
                    columns = ['event1','event2'])
lefth

,key1,key2,data
0,Ohio,2000,0.0
1,Ohio,2001,1.0
2,Ohio,2002,2.0
3,Nevada,2001,3.0
4,Nevada,2002,4.0


In [86]:
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

这种情况下，必须以列表的形式指明用作合并的多个列（注意用how=‘outer’对重复索引值的处理）

In [90]:
pd.merge(lefth,righth,left_on = ['key1','key2'],right_index = True)

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4,5
0,Ohio,2000,0.0,6,7
1,Ohio,2001,1.0,8,9
2,Ohio,2002,2.0,10,11
3,Nevada,2001,3.0,0,1


In [91]:
pd.merge(lefth,righth,left_on = ['key1','key2'],right_index = True,how = 'outer')

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4.0,5.0
0,Ohio,2000,0.0,6.0,7.0
1,Ohio,2001,1.0,8.0,9.0
2,Ohio,2002,2.0,10.0,11.0
3,Nevada,2001,3.0,0.0,1.0
4,Nevada,2002,4.0,NaN,NaN
4,Nevada,2000,NaN,2.0,3.0


同时使用合并双方的索引也没问题：

In [94]:
left2 = pd.DataFrame([[1.,2.],[3.,4.,],[5.,6.]],index = ['a','c','e'],columns = ['Ohio','Nevada'])
left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [96]:
right2 = pd.DataFrame([[7.,8.],[9.,10.],[11.,12.],[13.,14,]],index = ['b','c','d','e'],
                     columns = ['Missouri','Alabama'])
right2

,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [97]:
pd.merge(left2,right2,left_index = True,right_index = True,how = 'outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


DataFrame还有一个便捷的join实例方法，还可用于合并多个带有相同或相似索引的DataFrame对象，但要求没有重叠的列。

In [98]:
left2.join(right2,how = 'outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


DataFrame的join方法默认使用的是左连接，保留左边表的行索引！！！！它还支持在调用的DataFrame的列上，连接传递的DataFrame索引：

In [99]:
left1.join(right1,on = 'key1')   #左连接，保留left1的行索引，

,key1,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


对于简单的索引合并，还可以向join传入一组DataFrame：

In [100]:
another = pd.DataFrame([[7.,8.],[9.,10.],[11.,12.],[16.,17.]],index = ['a','c','e','f'],
                      columns = ['NY','Oregon'])
another

,NY,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [101]:
left2.join([right2,another])  #默认左连接，保留左列表的行索引！！！

,Ohio,Nevada,Missouri,Alabama,NY,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0


In [102]:
left2.join([right2,another],how = 'outer')

D:\Anaconda111\lib\site-packages\pandas\core\frame.py:6848: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  verify_integrity=True)


,Ohio,Nevada,Missouri,Alabama,NY,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
b,NaN,NaN,7.0,8.0,NaN,NaN
c,3.0,4.0,9.0,10.0,9.0,10.0
d,NaN,NaN,11.0,12.0,NaN,NaN
e,5.0,6.0,13.0,14.0,11.0,12.0
f,NaN,NaN,NaN,NaN,16.0,17.0


# 轴向连接

In [103]:
arr = np.arange(12).reshape((3,4))
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [104]:
np.concatenate([arr,arr],axis = 1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [105]:
np.concatenate([arr,arr],axis = 0)

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

pandas的concat函数

In [108]:
s1 = pd.Series([0,1],index = ['a','b'])
s1

a    0
b    1
dtype: int64

In [109]:
s2 = pd.Series([2,3,4],index = ['c','d','e'])
s2

c    2
d    3
e    4
dtype: int64

In [110]:
s3 = pd.Series([5,6],index = ['f','g'])
s3

f    5
g    6
dtype: int64

In [111]:
pd.concat([s1,s2,s3])    #pd.concat  默认是交集 即 how = ‘outer’

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

默认情况下，concat是在axis = 0上工作的，最终产生了一个新的Series。如果传入axis=1，则产生一个DataFrame：

In [112]:
pd.concat([s1,s2,s3],axis=1)

D:\Anaconda111\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


默认是外连接（有序并集），传入join = ‘inner’可得到它们的交集：

In [114]:
s4 = pd.concat([s1,s3])
s4

a    0
b    1
f    5
g    6
dtype: int64

In [117]:
pd.concat([s1,s4],axis = 1,join = 'inner') #join !!!

,0,1
a,0,0
b,1,1


可以通过join_axes指定要在其他轴上使用的索引，即指定索引

In [118]:
pd.concat([s1,s4],axis = 1,join_axes = [['a','c','b','e']])

,0,1
a,0.0,0.0
c,NaN,NaN
b,1.0,1.0
e,NaN,NaN


参与连接的片段在结果中区分不开。假设想要在连接轴上创建一个层次化索引，使用keys参数：

In [120]:
pd.concat([s1,s1,s3])

a    0
b    1
a    0
b    1
f    5
g    6
dtype: int64

In [121]:
result = pd.concat([s1,s1,s3],keys = ['one','two','three'])  #s1-one 、s2-two、s3-three
result

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: int64

In [122]:
result.unstack()  # unstack（）   解除层次化索引

,a,b,f,g
one,0.0,1.0,NaN,NaN
two,0.0,1.0,NaN,NaN
three,NaN,NaN,5.0,6.0


如果沿着axis = 1对 Series进行合并，则keys就会成为DataFrame的列头：

In [123]:
pd.concat([s1,s1,s3],axis=1,keys = ['one','two','three'])

D:\Anaconda111\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,one,two,three
a,0.0,0.0,NaN
b,1.0,1.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


DataFrame对象的例子：

In [125]:
df1 = pd.DataFrame(np.arange(6).reshape((3,2)),index = ['a','b','c'],columns = ['one','two'])
df1

,one,two
a,0,1
b,2,3
c,4,5


In [128]:
df2 = pd.DataFrame(np.arange(6,10).reshape((2,2)),index = ['a','c'],columns = ['three','four'])
df2

,three,four
a,6,7
c,8,9


In [129]:
pd.concat([df1,df2])

D:\Anaconda111\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,four,one,three,two
a,NaN,0.0,NaN,1.0
b,NaN,2.0,NaN,3.0
c,NaN,4.0,NaN,5.0
a,7.0,NaN,6.0,NaN
c,9.0,NaN,8.0,NaN


In [130]:
pd.concat([df1,df2],axis = 1)

D:\Anaconda111\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,one,two,three,four
a,0,1,6.0,7.0
b,2,3,NaN,NaN
c,4,5,8.0,9.0


In [131]:
pd.concat([df1,df2],axis = 1 ,keys = ['lv1','lv2'])  #通过keys参数来创建层次化参数

D:\Anaconda111\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


lv1       lv2     
  one two three four
a   0   1   6.0  7.0
b   2   3   NaN  NaN
c   4   5   8.0  9.0

如果传入的不是列表而是一个字典，则字典的键就会被当做keys选项的值：

In [135]:
pd.concat({'lv1':df1,'lv2':df2},axis=1)

D:\Anaconda111\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


lv1       lv2     
  one two three four
a   0   1   6.0  7.0
b   2   3   NaN  NaN
c   4   5   8.0  9.0

可以用nanmes参数来命名创建的轴级别（层次化索引）：

In [136]:
pd.concat([df1,df2],axis = 1,keys=['lv1','lv2'],names = ['upper','lower'])

D:\Anaconda111\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


upper lv1       lv2     
lower one two three four
a       0   1   6.0  7.0
b       2   3   NaN  NaN
c       4   5   8.0  9.0

DataFrame的行索引不包含任何相关数据：

In [138]:
df1 = pd.DataFrame(np.random.randn(3,4),columns = ['a','b','c','d'])
df1

,a,b,c,d
0,1.390604,1.245229,-0.233169,-1.525747
1,1.387266,0.059112,-0.396197,0.625664
2,-0.131517,-1.030650,0.266607,-0.198682


In [139]:
df2 = pd.DataFrame(np.random.randn(2,3),columns = ['b','d','a'])
df2

,b,d,a
0,-1.196291,0.635390,0.918831
1,-0.187209,-1.937727,0.562505


In [140]:
pd.concat([df1,df2])  #行索引是 0,1,2,0,1

D:\Anaconda111\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,a,b,c,d
0,1.390604,1.245229,-0.233169,-1.525747
1,1.387266,0.059112,-0.396197,0.625664
2,-0.131517,-1.030650,0.266607,-0.198682
0,0.918831,-1.196291,NaN,0.635390
1,0.562505,-0.187209,NaN,-1.937727


在这种情况下，传入ignore_index = True即可：

In [141]:
pd.concat([df1,df2],ignore_index=True)  #行索引变为 0,1,2,3,4

D:\Anaconda111\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,a,b,c,d
0,1.390604,1.245229,-0.233169,-1.525747
1,1.387266,0.059112,-0.396197,0.625664
2,-0.131517,-1.030650,0.266607,-0.198682
3,0.918831,-1.196291,NaN,0.635390
4,0.562505,-0.187209,NaN,-1.937727


# 合并重叠数据

索引全部或部分重叠

In [142]:
a =pd.Series([np.nan,2.5,np.nan,3.5,4.5,np.nan],index = ['f','e','d','c','b','a'])
a

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

In [146]:
b = pd.Series(np.arange(len(a),dtype=np.float64),index = ['f','e','d','c','b','a'])
b[-1]=np.nan
b

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    NaN
dtype: float64

In [148]:
np.where(pd.isnull(a),b,a)    #if isnull(a) True 则 b，else a

array([0. , 2.5, 2. , 3.5, 4.5, nan])

Series有一个combine_first方法，实现的也是一样的功能，还带有pandas的数据对齐：

In [149]:
b[:-2].combine_first(a[2:])

a    NaN
b    4.5
c    3.0
d    2.0
e    1.0
f    0.0
dtype: float64

对于DataFrame，combine_first看做：用传递对象中的数据为调用对象的缺失数据“打补丁”：

In [151]:
df1 = pd.DataFrame({'a':[1.,np.nan,5.,np.nan],'b':[np.nan,2.,np.nan,6.],'c':range(2,18,4)})
df1

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [152]:
df2 = pd.DataFrame({'a':[5.,4.,np.nan,3.,7.],'b':[np.nan,3.,4.,6.,8.]})
df2

,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [153]:
df1.combine_first(df2)  #在df1上打df2‘补丁’，即将df1中的NaN补成df2中对应的值，并将自身在df2中没有的数据补上

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


# 重塑和轴向旋转

有许多用于重新排列表格型数据的基础运算。这些函数也称作重塑（reshape）或轴向旋转（pivot）运算。

# 重塑层次化索引

层次化索引为DataFrame数据的重排任务提供了一种具有良好一致性的方式。主要功能有二：
1.satck（）：将数据的列‘旋转’为行   （生成层次化索引）
2.unstack（）：将数据的行‘旋转’为列   （解除层次化索引）

In [154]:
data = pd.DataFrame(np.arange(6).reshape((2,3)),index = pd.Index(['Ohio','Colorado'],name = 'state'),
                   columns = pd.Index(['one','two','three'],name = 'number'))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


对该数据使用stack（）方法即可将列转换为行，得到一个层次化索引的Series：

In [158]:
result=data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

对于一个层次化索引的Series，可以用unstack（）将其重排为一个DataFrame（解除层次化）：

In [159]:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


默认情况下，unstack（）操作的是最内层（stack（）也是如此）即保留最外层。传入分层级别的编号或名称即可对其他级别进行unstack（）操作：

In [160]:
result.unstack(0)    #unstack(0) 操作最外层，即将最外层的行转换成列

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [161]:
result.unstack('state')

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


如果不是所有的级别值都能在个分组找到的话，则unstack（）操作可能会引入缺失数据：

In [163]:
s1 = pd.Series([0,1,2,3],index = ['a','b','c','d'])
sd = pd.Series([4,5,6],index = ['c','d','e'])

In [164]:
s1

a    0
b    1
c    2
d    3
dtype: int64

In [165]:
sd

c    4
d    5
e    6
dtype: int64

In [172]:
data2 = pd.concat([s1,sd],keys=['one','two'])   # keys=[  ] 创建一个层次化索引
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [173]:
data2.unstack()  #默认操作最内层（保留最外层）

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


stack（）默认过滤缺失数据，该运算可逆：

In [174]:
data2.unstack().stack()

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [175]:
data2.unstack().stack(dropna= False)  #  不过滤缺失数据

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

在队DataFrame进行unstack（）操作时，作为旋转轴的级别将会成为结果中最低的级别：

In [177]:
df = pd.DataFrame({'left':result,'right':result+5},columns =pd.Index(['left','right'],name = 'side'))
df                  

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [179]:
df.unstack('state')  # unstack ('state')  被操作的state轴成为级别最低，即最内层

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

当调用stack（）时，可以指定轴的名字：

In [180]:
df.unstack('state').stack('side')  # 操作 ‘side’ 成为轴级别最低的即最内层

state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7

# 将“长格式”旋转成“宽格式” 

In [189]:
data = pd.read_csv('examples\\macrodata.csv')
data

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.980,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.150,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.350,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.370,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.540,139.6,3.50,5.2,180.007,2.31,1.19
5,1960.0,2.0,2834.390,1792.9,298.152,460.400,1966.1,29.550,140.2,2.68,5.2,180.671,0.14,2.55
6,1960.0,3.0,2839.022,1785.8,296.375,474.676,1967.8,29.750,140.9,2.36,5.6,181.528,2.70,-0.34
7,1960.0,4.0,2802.616,1788.2,259.764,476.434,1966.6,29.840,141.1,2.29,6.3,182.287,1.21,1.08
8,1961.0,1.0,2819.264,1787.7,266.405,475.854,1984.5,29.810,142.1,2.37,6.8,182.992,-0.40,2.77
9,1961.0,2.0,2872.005,1814.3,286.246,480.328,2014.4,29.920,142.9,2.29,7.0,183.691,1.47,0.81


In [190]:
data.head()

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [192]:
periods = pd.PeriodIndex(year=data.year,quarter = data.quarter,name = 'date')
columns = pd.Index(['realgdp','infl','unemp'],name = 'item')
data = data.reindex(columns = columns)
data.index = periods.to_timestamp('D','end')
ldata = data.stack().reset_index().rename(columns = {0:'value'})
pivoted = ldata.pivot('date','item','value')
pivoted

AttributeError: 'DataFrame' object has no attribute 'year'